# Notes
https://access.redhat.com/solutions/4795571

In [12]:
# imports
import base64
import json
import subprocess
import tempfile
from pathlib import Path
from pprint import pprint
from shutil import make_archive, rmtree
from urllib.parse import quote, quote_plus
from uuid import uuid4

import requests


In [13]:
# settings

namespace = "openshift-monitoring"

In [14]:
# getting secret
# secret = subprocess.run(["oc", "get", "secret", "-n", namespace, "|", "grep",  "prometheus-k8s-token", "|", "head", "-n1", "|", "awk", "'{ print $1 }'"])
data = subprocess.run(["oc", "get", "secret", "-n", namespace], capture_output=True)
data = data.stdout.decode()
data = data.split("\n")
secret = None
for d in data:
    a = d.split(" ")
    if a[0].startswith("prometheus-k8s-token"):
        secret = a[0]
        break


print(secret)

prometheus-k8s-token-d2cc6


In [15]:
# getting token
data1 = subprocess.run(["oc", "get", "secret", secret, "-n", namespace, "-o", "jsonpath='{.data.token}" ], capture_output=True)
data1 = data1.stdout.decode()
token = base64.b64decode(data1).decode()

print(token)


eyJhbGciOiJSUzI1NiIsImtpZCI6IkRBTV9ndUEyQmZSOEdWSUltY3lOZll2LXdaWUtPMzlpTzNKekRvVDc5Z2MifQ.eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJvcGVuc2hpZnQtbW9uaXRvcmluZyIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VjcmV0Lm5hbWUiOiJwcm9tZXRoZXVzLWs4cy10b2tlbi1kMmNjNiIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50Lm5hbWUiOiJwcm9tZXRoZXVzLWs4cyIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50LnVpZCI6ImU2MzM0ZjVmLTBlYWEtNGI3OS04MjU0LTAwOTllOTIyNDZiMSIsInN1YiI6InN5c3RlbTpzZXJ2aWNlYWNjb3VudDpvcGVuc2hpZnQtbW9uaXRvcmluZzpwcm9tZXRoZXVzLWs4cyJ9.okh2ez5IjlFEGQ5kgn2rZwDm2gJS-TZuznA2cqhtauoC2RnzWP8pU7KwmAr7fFfjNCosDxGB2yEJV1HbXdgJG2XKkGJH4bXWCMGeXmHEN3j-hxzcgzfb5S8qiTV3tYG1XQDv-etuurw4Aw3sI7F4Om89rEEzfTSFwNLdn3c_Wh1AtxEIosQm6mqKSZRbDPTBK6QKJcUNSonqwk_J_w0_U99urP1W7SdMvOc_cmqQyi8fScfR49ikclJrOnP7HBB7EpgqcKe1TgSW1x-ZsV9EWvkQOjgYN1rdPaO5JycK1PiMSkK0QFGfx2Y82Onhx-RJ2iT2sLD7u3eAs5hiwyJmOg


In [16]:
# getting prometheus host
data2 = subprocess.run(["oc", "get", "route", "prometheus-k8s", "-n", namespace, "-o", "jsonpath={.spec.host}" ], capture_output=True)
host = data2.stdout.decode()
print(host)

prometheus-k8s-openshift-monitoring.apps.eng-long-lived.9p7x.s1.devshift.org


In [17]:
# query the api

# get container memory
query = "max by(container) (container_memory_usage_bytes{namespace=~'.+rhoam.+'})"
query = quote_plus(query)
r1 = requests.get(
    f'https://{host}/api/v1/query?query={query}',
    headers={"Authorization": f"Bearer {token}"}
)


# get container cpu
query = 'max_over_time(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate{namespace=~".+rhoam.+"}[1y])'
query = quote_plus(query)
r2 = requests.get(
    f'https://{host}/api/v1/query?query={query}',
    headers={"Authorization": f"Bearer {token}"}
)


In [18]:

container_memory = r1.json()
container_cpu = r2.json()

pprint(container_memory)
pprint(container_cpu)

{'data': {'result': [{'metric': {'container': 'POD'},
                      'value': [1664806426.227, '0']},
                     {'metric': {'container': 'alertmanager'},
                      'value': [1664806426.227, '24858624']},
                     {'metric': {'container': 'apicast-production'},
                      'value': [1664806426.227, '43290624']},
                     {'metric': {'container': 'apicast-staging'},
                      'value': [1664806426.227, '45432832']},
                     {'metric': {'container': 'backend-cron'},
                      'value': [1664806426.227, '40058880']},
                     {'metric': {'container': 'backend-listener'},
                      'value': [1664806426.227, '507940864']},
                     {'metric': {'container': 'backend-worker'},
                      'value': [1664806426.227, '51589120']},
                     {'metric': {'container': 'blackbox-exporter'},
                      'value': [1664806426.227, '24580096

In [19]:
# query the api

# get namespace memory
query = "max by(namespace) (container_memory_usage_bytes{namespace=~'.+rhoam.+'})"
query = quote_plus(query)
r3 = requests.get(
    f'https://{host}/api/v1/query?query={query}',
    headers={"Authorization": f"Bearer {token}"}
)


# get namespace cpu
query = "max_over_time (namespace:container_cpu_usage:sum{namespace=~'.+rhoam.+'}[1y])"
query = quote_plus(query)
r4 = requests.get(
    f'https://{host}/api/v1/query?query={query}',
    headers={"Authorization": f"Bearer {token}"}
)

In [20]:
namespace_memory = r3.json()
namespace_cpu = r4.json()

pprint(namespace_memory)
pprint(namespace_cpu)

{'data': {'result': [{'metric': {'namespace': 'redhat-rhoami-3scale'},
                      'value': [1664806426.623, '2149240832']},
                     {'metric': {'namespace': 'redhat-rhoami-marin3r-operator'},
                      'value': [1664806426.623, '231636992']},
                     {'metric': {'namespace': 'redhat-rhoami-rhsso-operator'},
                      'value': [1664806426.623, '118919168']},
                     {'metric': {'namespace': 'redhat-rhoami-cloud-resources-operator'},
                      'value': [1664806426.623, '107192320']},
                     {'metric': {'namespace': 'redhat-rhoami-observability'},
                      'value': [1664806426.623, '3561537536']},
                     {'metric': {'namespace': 'redhat-rhoami-3scale-operator'},
                      'value': [1664806426.623, '52531200']},
                     {'metric': {'namespace': 'redhat-rhoami-user-sso-operator'},
                      'value': [1664806426.623, '125435904']}

In [21]:
from subprocess import CalledProcessError

# get the rhmi cr for the cluster

# cr = ! oc get rhmi rhoam -n redhat-rhoam-operator -o json
run = True
count = 0
namespaces = ["redhat-rhoam-operator", "redhat-rhoami-operator", "local-rhoam-operator"]
cr = None
while run and count < len(namespaces):
    run = False
    try:
        cmd = subprocess.run(["oc", "get", "rhmi", "rhoam", "-n", namespaces[count], "-o", "json"], capture_output=True, check=True)
        data = cmd.stdout.decode()
        cr = json.loads(data)
    except CalledProcessError:
        print(f"failed to find namespace: {namespaces[count]}")
        run = True
        count += 1

addon = {
        'version': "",
        'quota': ""
    }

if cr is None:
    print("Failed to find RHOAM CR")
else:
    addon = {
        'version': cr['status']['version'],
        'quota': cr['status']['quota']
    }
    pprint(addon)


failed to find namespace: redhat-rhoam-operator
{'quota': '100K', 'version': '1.26.0'}


In [22]:
# Save results to folder

data_location = Path(tempfile.gettempdir(), "metric_collection")
result_folder = Path(data_location, str(uuid4()))
result_folder.mkdir(parents=True, exist_ok=False)

container_memory_file = Path(result_folder, "container_memory.json")
container_cpu_file = Path(result_folder, "container_cpu.json")
namespace_memory_file = Path(result_folder, "namespace_memory.json")
namespace_cpu_file = Path(result_folder, "namespace_cpu.json")
addon_file = Path(result_folder, "addon.json")

files = [container_memory_file, container_cpu_file, namespace_memory_file, namespace_cpu_file, addon_file]

with open(container_memory_file, "w") as f:
    f.write(json.dumps(container_memory))

with open(container_cpu_file, "w") as f:
    f.write(json.dumps(container_cpu))

with open(namespace_memory_file, "w") as f:
    f.write(json.dumps(namespace_memory))

with open(namespace_cpu_file, "w") as f:
    f.write(json.dumps(namespace_cpu))

with open(addon_file, "w") as f:
    f.write(json.dumps(addon))

make_archive(result_folder, "zip", result_folder)
rmtree(result_folder)